In [1]:
# use this as as base for CNN model
# https://www.kaggle.com/sunnyguha/basic-cnn-starter

In [1]:
# import images - 1000 to begin with and on a lazy basis
#   need 1000 images that correspond to 1000 y_train
#   images as np.array(1000,size,size,3) and np.array(1000,1)
#   user Keras flow from directory

# CNN Model
#   Layers
#   Compile
#     During compile stage, set the parameter that you want to minimize
#     Weighted column


# View results

# Check
#   In the case of false positives for a class, is it more likely that the result is of a different class or that it is negative in all classes


In [2]:
import pandas as pd
from tensorflow.python.keras.utils.data_utils import Sequence

In [12]:
# Set global variables
# un-comment as they are used
# batch_size=32
# validation_ratio=0.1
# sample_size=2000
# epochs=3
# img_size=512
data_path = "../data/"   # used when calling DataGenerator

In [14]:
train = pd.read_csv(data_path + 'stage_1_train.csv')
train['filename'] = train['ID'].apply(lambda x: "ID_" + x.split('_')[1] + ".dcm")


read_data = train.copy()
train.head()

,ID,Label,filename
0,ID_63eb1e259_epidural,0,ID_63eb1e259.dcm
1,ID_63eb1e259_intraparenchymal,0,ID_63eb1e259.dcm
2,ID_63eb1e259_intraventricular,0,ID_63eb1e259.dcm
3,ID_63eb1e259_subarachnoid,0,ID_63eb1e259.dcm
4,ID_63eb1e259_subdural,0,ID_63eb1e259.dcm


In [ ]:
train.label.head()

DataGenerator class based on code written by https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly.  The DataGenerator class inherits from the Keras utils.Sequence class and is used to read and serve the train data in batches.

In [7]:
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs_labels, data_path = '', batch_size=100, dim=(512,512)):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs_labels
        self.data_path = data_path
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs['filename'][k] for k in indexes]
        list_label_temp=[[int(self.list_IDs['any'][i]),int(self.list_IDs['epidural'][i]),int(self.list_IDs['intraparenchymal'][i]),int(self.list_IDs['intraventricular'][i]),int(self.list_IDs['subarachnoid'][i]),int(self.list_IDs['subdural'][i])] for i in indexes]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp,list_label_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))

    def __data_generation(self, list_IDs_temp,list_label_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = []
        y = []

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            ds=pydicom.dcmread(self.data_path+'stage_1_train_images/' +list_IDs_temp[i] )
            temp=ds.pixel_array
            window_center , window_width, intercept, slope = get_windowing(ds)
            img = window_image(temp, 50, 100, intercept, slope)
            resized = cv2.resize(img, (200, 200))
            X.append(resized)       
        X=np.array(X).reshape(-1,200,200,1)
        y_train=np.asarray(list_label_temp) 
        return X,y_train